In [7]:
data Shape = Circle | Square deriving (Show, Eq, Ord )

In [4]:
Circle

Circle

In [28]:
import Linear

In [30]:
V2 2 3

V2 2 3

In [13]:
data Expr = X | Lit Double | Times Expr Expr | Plus Expr Expr deriving Show

-- data Var a = X | Lit a
-- data Num a = Lit a | ...


In [14]:

interp X x = x
interp (Times a b) x = (interp a x) * (interp b x)
interp (Plus a b) x = (interp a x) + (interp b x) 
interp (Lit a) x = a

Line 3: Redundant bracket
Found:
(interp a x) * (interp b x)
Why not:
interp a x * (interp b x)Line 3: Redundant bracket
Found:
(interp a x) * (interp b x)
Why not:
(interp a x) * interp b xLine 4: Redundant bracket
Found:
(interp a x) + (interp b x)
Why not:
interp a x + (interp b x)Line 4: Redundant bracket
Found:
(interp a x) + (interp b x)
Why not:
(interp a x) + interp b x

In [15]:

diff X = Lit 1
diff (Times a b) = Plus (Times (diff a) b) (Times a (diff b))
diff (Plus a b) = Plus (diff a) (diff b)
diff (Lit _) = Lit 0

In [17]:
diff (Times X (Plus X X))

Plus (Times (Lit 1.0) (Plus X X)) (Times X (Plus (Lit 1.0) (Lit 1.0)))

In [ ]:
data ExprD = X | Diff Expr | Plus | Times | Lit

-- data FreeDiff x = Diff x | Pure x

interpd :: ExprD -> Double -- interpd = interp (lower e) 
lower :: ExprD -> Expr
lower (Diff x) = diff (lower x)
lower X = X
lower (Times x y ) = Times (lower x) (lower y)





interp . diff

interp (diff X) = interp (Lit 1) = \_ -> 1
interp (diff (Times a b)) = 
interp (diff (Plus a b)) = 
interp (diff (Lit n)) = interp (Lit 0) = \x -> 0



In [21]:
simplify (Times (Lit 0) x) = Lit 0
simplify (Times (Lit 1) x) = simplify x -- and vice versa
simplify (Plus (Lit n) (Lit m)  ) = Lit (n + m)
simplify  (Times (Lit n) (Lit m)  ) = Lit (n * m)
simplify (Plus x y) = Plus (simplify x) (simplify y)
simplify (Times x y) = Times (simplify x) (simplify y)
simplify x = x


simplify (diff (Times X (Plus X X))) -- that's a little better

Plus (Plus X X) (Times X (Lit 2.0))

In [24]:
:set -XFlexibleInstances

In [27]:
-- interp . diff -- look at this thing inlined.

class ExprF a where
   x :: a
   times :: a -> a -> a
   plus :: a -> a -> a
   lit :: Double -> a

instance ExprF (Double -> Double) where -- ExprF (a -> a) would be fun
   x y = y
   times f g x = (f x) * (g x) 
   plus f g x = (f x) + (g x)
   lit a _ = a
   


Line 11: Redundant bracket
Found:
(f x) * (g x)
Why not:
f x * (g x)Line 11: Redundant bracket
Found:
(f x) * (g x)
Why not:
(f x) * g xLine 12: Redundant bracket
Found:
(f x) + (g x)
Why not:
f x + (g x)Line 12: Redundant bracket
Found:
(f x) + (g x)
Why not:
(f x) + g x

In [22]:
newtype Diff = Diff (Double -> Double)

instance ExprF Diff where
  x _ = 1
  plus (Diff f) (Diff g) = Diff (\x -> f x + g x)
  times (Diff f) (Diff g) = Diff (\x -> let Ivf,df) = (f x) in let (vg, dg) = g x in ( vf * vg ,  df * vg + vf * dg))

: 

In [ ]:
newtype Diff = Diff (Double -> (Double, Double)  )

In [37]:
newtype Diff = Diff (Double -> (Double, Double))

instance ExprF Diff where
  x = Diff $ \z -> (x z, 1)
  plus (Diff f) (Diff g) = Diff (\x -> let (vf,df) = f x in let (vg, dg) = g x in (vf + vg, df + dg))
  times (Diff f) (Diff g) = Diff (\x -> let (vf,df) = f x in let (vg, dg) = g x in ( vf * vg ,  df * vg + vf * dg))
  lit n = Diff (\z -> (n, 0))

Line 7: Use const
Found:
\ z -> (n, 0)
Why not:
const (n, 0)

In [38]:
applydiff (Diff f) x = f x


Line 1: Eta reduce
Found:
applydiff (Diff f) x = f x
Why not:
applydiff (Diff f) = f

In [42]:
applydiff (plus (lit 3) $ times x x) 1

(4.0,2.0)